Import Libraries

In [16]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [17]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [18]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [19]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [20]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

In [21]:
conn

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [22]:
# Get the data values
data = tuple(stations.iloc[21].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [23]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (3687, 'Boardwalk West', 'active', '300 E. Riverside Dr.', '', 16683.0, 'parkland', 9.0, 'solar', 30.0, 5.0, '', 9, '2021-01-04T12:00:00Z')



In [ ]:
# Execute the query 
conn.execute(query)

In [ ]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [49]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [50]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[5].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [ ]:
result

**TASK: Create a Function to insert data for `trips` table**

In [24]:
data2 = trips.iloc[0]
tuple(data2.values)

(23455589,
 'Local365',
 '174',
 '2021-01-26 17:47:42 UTC',
 4059.0,
 'Nash Hernandez/East @ RBJ South',
 3660.0,
 'East 6th/Medina',
 7)

In [25]:
# Get the data values
data2 = tuple(trips.iloc[0].fillna('').values)

# Make the query
query = f"""
INSERT INTO trips
VALUES {data2}
"""

In [26]:
# See the actual query looks like
print(query)


INSERT INTO trips
VALUES (23455589, 'Local365', '174', '2021-01-26 17:47:42 UTC', 4059.0, 'Nash Hernandez/East @ RBJ South', 3660.0, 'East 6th/Medina', 7)



In [ ]:
# Execute the query 
conn.execute(query)

In [28]:
# Commit changes to database
conn.commit()

In [29]:
def insert_into_trips(data2, conn):
    query = f"""INSERT INTO trips values {data2}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [30]:
# Example use of the function 
conn = make_connection()
data2 = tuple(trips.iloc[1].fillna('').values) # Randomly select a data
result = insert_into_trips(data2, conn)

In [ ]:
result

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [ ]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [ ]:
print(query_station_id)

In [ ]:
# Run the query and get the result
result = pd.read_sql_query(query_station_all, conn)

In [ ]:
result

Just like the previous section, create a function to read specific station

In [52]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

In [53]:
get_station_id(2500,conn)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Create a Function to read data from `trips` table**

In [54]:
get_all_stations(conn)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z
2,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,,,,,,,,,9,2021-01-04T12:00:00Z
3,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
4,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
5,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,,,,,,,,,1,2021-01-04T12:00:00Z
6,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,,,,,,,,,9,2021-01-04T12:00:00Z
7,2502,Barton Springs & Riverside,closed,400 Barton Springs Rd.,,,,,,,,moved to Pedernales/2nd. Original city ID: 16729,9,2021-01-04T12:00:00Z
8,3687,Boardwalk West,active,300 E. Riverside Dr.,,16683,parkland,9,solar,30.0,5.0,,9,2021-01-04T12:00:00Z


In [32]:
# Get Specific trips ID  Query
id = 23455589
query_id = f"""
SELECT * FROM trips
WHERE id = {id}
"""

# Get all station ID Query
query_trips_all = "SELECT * FROM trips"

In [33]:
print(query_id)


SELECT * FROM trips
WHERE id = 23455589



In [34]:
# Run the query and get the result
result = pd.read_sql_query(query_trips_all, conn)

In [35]:
result

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
...,...,...,...,...,...,...,...,...,...
1342061,23515345,Local365,19476,2021-02-13 14:17:15 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,62
1342062,23553979,Local365,19418,2021-02-26 18:12:13 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,4
1342063,23499655,U.T. Student Membership,21553,2021-02-07 17:15:29 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,3
1342064,23538444,U.T. Student Membership,2147,2021-02-22 20:52:06 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,704


In [36]:
def get_id(id, conn):
    query = f"""SELECT * FROM trips WHERE id = {id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

In [37]:
get_id(23455589,conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059,Nash Hernandez/East @ RBJ South,3660,East 6th/Medina,7


In [38]:
get_all_trips(conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
...,...,...,...,...,...,...,...,...,...
1342061,23515345,Local365,19476,2021-02-13 14:17:15 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,62
1342062,23553979,Local365,19418,2021-02-26 18:12:13 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,4
1342063,23499655,U.T. Student Membership,21553,2021-02-07 17:15:29 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,3
1342064,23538444,U.T. Student Membership,2147,2021-02-22 20:52:06 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,704


# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [ ]:
# your code
from flask import Flask, request
import sqlite3
import requests
from tqdm import tqdm
import json 
import numpy as np
import pandas as pd


app = Flask(__name__) 


@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()

################ Functions ################


def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection


if __name__ == '__main__':
    app.run(debug=True, port=5000)

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [39]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [40]:
res

<Response [404]>

In [58]:
res.text

'{"station_id":{"0":2500,"1":2546,"2":2550,"3":2536,"4":2538,"5":2541,"6":2545,"7":2502,"8":3687},"name":{"0":"Republic Square","1":"ACC - West & 12th Street","2":"Republic Square @ Guadalupe & 4th St.","3":"Waller & 6th St.","4":"Bullock Museum @ Congress & MLK","5":"State Capitol @ 14th & Colorado","6":"ACC - Rio Grande & 12th","7":"Barton Springs & Riverside","8":"Boardwalk West"},"status":{"0":"closed","1":"closed","2":"closed","3":"closed","4":"closed","5":"closed","6":"closed","7":"closed","8":"active"},"address":{"0":"425 W 4th Street","1":"1231 West Ave.","2":"Presented by Austin Ventures","3":"602 Waller St.","4":"1881 Congress Ave.","5":"206 W. 14th St.","6":"700 W. 12th St.","7":"400 Barton Springs Rd.","8":"300 E. Riverside Dr."},"alternate_name":{"0":"","1":"","2":"","3":"","4":"","5":"","6":"","7":"","8":""},"city_asset_number":{"0":"","1":"","2":"","3":"","4":"","5":"","6":"","7":"","8":16683},"property_type":{"0":"","1":"","2":"","3":"","4":"","5":"","6":"","7":" ","8":

In [59]:
res.json()

{'station_id': {'0': 2500,
  '1': 2546,
  '2': 2550,
  '3': 2536,
  '4': 2538,
  '5': 2541,
  '6': 2545,
  '7': 2502,
  '8': 3687},
 'name': {'0': 'Republic Square',
  '1': 'ACC - West & 12th Street',
  '2': 'Republic Square @ Guadalupe & 4th St.',
  '3': 'Waller & 6th St.',
  '4': 'Bullock Museum @ Congress & MLK',
  '5': 'State Capitol @ 14th & Colorado',
  '6': 'ACC - Rio Grande & 12th',
  '7': 'Barton Springs & Riverside',
  '8': 'Boardwalk West'},
 'status': {'0': 'closed',
  '1': 'closed',
  '2': 'closed',
  '3': 'closed',
  '4': 'closed',
  '5': 'closed',
  '6': 'closed',
  '7': 'closed',
  '8': 'active'},
 'address': {'0': '425 W 4th Street',
  '1': '1231 West Ave.',
  '2': 'Presented by Austin Ventures',
  '3': '602 Waller St.',
  '4': '1881 Congress Ave.',
  '5': '206 W. 14th St.',
  '6': '700 W. 12th St.',
  '7': '400 Barton Springs Rd.',
  '8': '300 E. Riverside Dr.'},
 'alternate_name': {'0': '',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': '',
  '6': '',
  '7': '',


The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [60]:
pd.DataFrame(res.json())#.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z
2,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,,,,,,,,,9,2021-01-04T12:00:00Z
3,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
4,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
5,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,,,,,,,,,1,2021-01-04T12:00:00Z
6,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,,,,,,,,,9,2021-01-04T12:00:00Z
7,2502,Barton Springs & Riverside,closed,400 Barton Springs Rd.,,,,,,,,moved to Pedernales/2nd. Original city ID: 16729,9,2021-01-04T12:00:00Z
8,3687,Boardwalk West,active,300 E. Riverside Dr.,,16683,parkland,9,solar,30.0,5.0,,9,2021-01-04T12:00:00Z


In [41]:
url = 'http://127.0.0.1:5000/trips/'
res = requests.get(url)

In [42]:
res

<Response [404]>

In [ ]:
res.text

In [ ]:
res.text

In [ ]:
res.json()

## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [6]:
station_id = 2500
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [7]:
url

'http://127.0.0.1:5000/stations/2500'

In [ ]:
response = requests.get(url)
pd.DataFrame(response.json())

**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

In [ ]:
#your code
from flask import Flask, request
import sqlite3
import requests
from tqdm import tqdm
import json 
import numpy as np
import pandas as pd

app = Flask(__name__) 

@app.route('/')
@app.route('/homepage')
def home():
    return 'Hello World'

@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()

@app.route('/trips/<id>') # rute dinamis
def route_id(id):
    conn = make_connection()
    trips = get_id(id, conn)
    return trips.to_json()


################ Functions ################


def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

def get_id(id, conn):
    query = f"""SELECT * FROM trips WHERE id = {id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection


if __name__ == '__main__':
    app.run(debug=True, port=5000)

In [9]:
id = 9900288008
url1 = f"http://127.0.0.1:5000/trips/{id}"

In [10]:
url1

'http://127.0.0.1:5000/trips/9900288008'

In [ ]:
response = requests.get(url1)
pd.DataFrame(response.json())

In [ ]:
response = requests.get(url)
pd.DataFrame(response.json())

## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json', methods=['POST']) 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [2]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [3]:
res = requests.get(url, json=data)

In [4]:
res = requests.post(url, json=data)

In [5]:
res

<Response [200]>

In [6]:
res.text

'Hello Andi, your age is 24, and your address in my adress\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

In [ ]:
stations.iloc[25].fillna('').to_disct()

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [102]:
data = stations.iloc[25].fillna('').to_dict()

In [81]:
json.dumps(data)

'{"station_id": 3687, "name": "Boardwalk West", "status": "active", "address": "300 E. Riverside Dr.", "alternate_name": "", "city_asset_number": 16683.0, "property_type": "parkland", "number_of_docks": 9.0, "power_type": "solar", "footprint_length": 30.0, "footprint_width": 5.0, "notes": "", "council_district": 9, "modified_date": "2021-01-04T12:00:00Z"}'

In [14]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [104]:
data = stations.iloc[25].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [105]:
data_json

'{"station_id": 2574, "name": "Zilker Park", "status": "active", "address": "Barton Springs Rd", "alternate_name": "Zilker Park at Barton Springs and William Burton Drive", "city_asset_number": 16759.0, "property_type": "parkland", "number_of_docks": 17.0, "power_type": "solar", "footprint_length": 50.0, "footprint_width": 5.0, "notes": "parkland at ROW/easement", "council_district": 8, "modified_date": "2021-01-04T12:00:00Z"}'

In [106]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [107]:
res

<Response [200]>

In [108]:
res.text

'Error'

In [78]:
stations.iloc[15]

station_id                              1004
name                 Red River & LBJ Library
status                                closed
address                2322 Red River Street
alternate_name                           NaN
city_asset_number                        NaN
property_type                            NaN
number_of_docks                          NaN
power_type                               NaN
footprint_length                         NaN
footprint_width                          NaN
notes                                    NaN
council_district                           1
modified_date           2021-01-04T12:00:00Z
Name: 15, dtype: object

In [ ]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table
for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [83]:
trips.iloc[15]

trip_id                              23421503
subscriber_type                      Local365
bikeid                                  16288
start_time            2021-01-16 18:56:43 UTC
start_station_id                       3660.0
start_station_name            East 6th/Medina
end_station_id                         4050.0
end_station_name                 5th/Campbell
duration_minutes                           21
Name: 15, dtype: object

In [20]:
data = trips.iloc[25].fillna('').to_dict()

In [ ]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

In [24]:
json.dumps(data)

'{"trip_id": 23459963, "subscriber_type": "Local365", "bikeid": "16337", "start_time": "2021-01-28 08:06:10 UTC", "start_station_id": 4048.0, "start_station_name": "South Congress @ Bouldin Creek", "end_station_id": 3390.0, "end_station_name": "6th/Brazos", "duration_minutes": 6}'

In [22]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [25]:
data = trips.iloc[55].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [26]:
data_json

'{"trip_id": 23395380, "subscriber_type": "Local31", "bikeid": "938", "start_time": "2021-01-08 14:02:43 UTC", "start_station_id": 4054.0, "start_station_name": "Rosewood/Chicon", "end_station_id": 4047.0, "end_station_name": "8th/Lavaca", "duration_minutes": 11}'

In [97]:
url = "http://127.0.0.1:5000/trips/add"
res = requests.post(url, json=data_json)

In [98]:
res

<Response [200]>

In [99]:
res.text

'Error'

In [ ]:
# Sintaks for iteratively insert trips (from trips_csv) into stations table
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

In [13]:
res.text

'OK'

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 2 points - Created functionality to read or get specific data from the database
- [x] 4 points - Created functionality to input new data into each table for the databases
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

**Your Code Here**

```python
# example of analytical static endpoint
@app.route('/trips/average_duration') 
    # your code
```

In [ ]:
@app.route('/trips/average_duration') 


In [ ]:
@app.route('/trips/average_duration/<bikeid>') 

**TEST YOUR ENDPOINT**

After implement it to `app.py` file, **test and implement** your **static endpoint** with the following cells.

In [88]:
url = 'http://127.0.0.1:5000/trips/average_duration'
res = requests.get(url)
res

<Response [200]>

In [45]:
# show the result as dataframe
pd.DataFrame(res.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
...,...,...,...,...,...,...,...,...,...
1342061,23515345,Local365,19476,2021-02-13 14:17:15 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,62
1342062,23553979,Local365,19418,2021-02-26 18:12:13 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,4
1342063,23499655,U.T. Student Membership,21553,2021-02-07 17:15:29 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,3
1342064,23538444,U.T. Student Membership,2147,2021-02-22 20:52:06 UTC,3838,26th/Nueces,3841,23rd/Rio Grande,704


## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

**Your Code Here**

```python
# example of analytical dynamic endpoint
@app.route('/trips/average_duration/<bike_id>')
    # code here
```

**TEST YOUR ENDPOINT**

After implement it to `app.py` file, **test and implement** your **dynamic endpoint** with the following cells.

In [ ]:
# test dynamic endpoint
url = 'http://127.0.0.1:5000/<your_endpoint>/<dynamic_endpoint>'
res = requests.get(url)
res

In [ ]:
# test dynamic endpoint
url = 'http://127.0.0.1:5000/trips/average_duration/<duration_minutes>'
res = requests.get(url)
res

In [ ]:
response = request.get(url)
pd.DataFrame(response.json)

In [ ]:
res.text

In [ ]:
# show the result as dataframe
pd.DataFrame(res.json())

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

After created the endpoint, make sure to implement it to app.py file.

In [ ]:
# Your Code Here
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['2015-08'] # Select specific items (period) from the dictionary (the value will be "2015-08")


**TEST YOUR ENDPOINT**

After implement it to `app.py` file, **test and implement** your **POST endpoint** with the following cells.

In [ ]:
url = "http://127.0.0.1:5000/trips/add"
res = requests.post(url, json=data_json)

In [ ]:
input_data = request.get_json() # Get the input as dictionary


In [ ]:
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

In [ ]:
# test POST endpoint
data = {
    # input data
}

# post endpoint
url = "http://127.0.0.1:5000/<POST_endpoint>"
res = requests.post(url, json = data)

# show code result
res

In [ ]:
# show the result as dataframe
pd.DataFrame(res.json())

In [84]:
data = {
    "id" : "8997555", 
    "bikeid" : 28,
    "duration_minutes" : "142 Hours"
}

In [85]:
# post endpoint
url = "http://127.0.0.1:5000/json"
res = requests.post(url, json = data)

In [86]:
res

<Response [200]>

In [87]:
res.text

'Hello 8997555, your bikeid is 28, and the length of your cycling time is 142 Hours\n            '

# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. Make sure you have implemented all the endpoints to `app.py` (including your custom static, dynamic and post endpoints).
3. Submit your `Bikeshare API.ipynb` and `app.py` file in your github repository. The dataset is optional to post since it has big size to post. 

